# 学習モデルを使って実際のレース予想をする

In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

### 予想したいレースのURLを以下に入力　ここのサイトから選ぶ　https://keirin.kdreams.jp/

In [64]:
url = "https://keirin.kdreams.jp/hofu/racedetail/6320210906030011/"

In [65]:
columns = ['予想', '好気合', '総評', '枠番', '車番', '選手名府県/年齢/期別', '級班', '脚質', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外']

### データを取ってくる

In [66]:
race_data = pd.read_html(url)[4][:-1]
race_data.columns = columns
race_data = race_data.astype(str)
time.sleep(1)

### データを加工する

In [67]:
race_data = pd.concat([race_data, race_data["選手名府県/年齢/期別"].str.split("/", expand=True)], axis=1).drop("選手名府県/年齢/期別", axis=1)
race_data = race_data.drop(0, axis=1)

race_data["予想"] = pd.Categorical(race_data["予想"], categories=["nan", "×", "▲", "△", "○", "◎", "注"])
race_data["好気合"] = pd.Categorical(race_data["好気合"], categories=["★"])
race_data["脚質"] = pd.Categorical(race_data["脚質"], categories=["両", "追", "逃"])
race_data["級班"] = pd.Categorical(race_data["級班"], categories=["A1", "A2", "A3", "L1", "S1", "S2", "SS"])

prediction = pd.get_dummies(race_data["予想"])
isFine = pd.get_dummies(race_data["好気合"])
leg_type = pd.get_dummies(race_data["脚質"])
group = pd.get_dummies(race_data["級班"])

race_data = pd.concat([race_data, prediction, isFine, leg_type, group], axis=1).drop(["予想", "好気合", "脚質", "級班"], axis=1)
print(race_data.columns)
race_data = race_data.replace(["失", "落", "故", "欠"], 9)
race_data["ギヤ倍数"] = race_data["ギヤ倍数"].map(lambda x: x[:4] if len(x)>4 else x)
race_data[2] = race_data[2].map(lambda x: x.replace(" （欠車）", "") if "欠車"in x else x)
race_data = race_data.astype("float64")
race_data = race_data.loc[:, ['車番', '総評', '枠番', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外', 1, 2, 'nan', '×', '▲', '△', '○', '◎', '注', '★', '両', '追', '逃', 'A1', 'A2', 'A3', 'L1', 'S1', 'S2', 'SS']]
race_data

Index([  '総評',   '枠番',   '車番', 'ギヤ倍数', '競走得点',   '1着',   '2着',   '3着',   '着外',
            1,      2,  'nan',    '×',    '▲',    '△',    '○',    '◎',    '注',
          '★',    '両',    '追',    '逃',   'A1',   'A2',   'A3',   'L1',   'S1',
         'S2',   'SS'],
      dtype='object')


,車番,総評,枠番,ギヤ倍数,競走得点,1着,2着,3着,着外,1,...,両,追,逃,A1,A2,A3,L1,S1,S2,SS
0,1.0,2.0,1.0,3.92,93.34,1.0,0.0,0.0,2.0,38.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,2.0,3.92,92.33,1.0,3.0,1.0,1.0,44.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2.0,3.0,3.92,91.19,0.0,0.0,0.0,0.0,23.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4.0,4.0,4.0,3.92,83.20,0.0,0.0,0.0,0.0,26.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5.0,3.0,5.0,3.86,88.05,0.0,1.0,3.0,2.0,46.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6.0,4.0,6.0,3.92,81.70,0.0,2.0,0.0,7.0,22.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,7.0,1.0,6.0,3.92,96.42,3.0,0.0,0.0,0.0,33.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 学習済みモデルに入れて予想する

In [68]:
import tensorflow as tf
from tensorflow import keras

In [69]:
model = tf.keras.models.load_model("pred_top3")

In [70]:
race_data.shape

(7, 29)

In [92]:
results = model.predict(race_data)
results

array([[0.7240775 ],
       [0.3984066 ],
       [0.4925891 ],
       [0.8068186 ],
       [0.2931009 ],
       [0.18429533],
       [0.57692647]], dtype=float32)

In [93]:
import numpy as np
def printResult(results):
    new_results = []
    for result in results:
        new_results.append(result[0])
        
    sorted_result = np.argsort(new_results)
    sorted_result = list(map(lambda x: x+1, sorted_result))
    new_results = sorted(new_results)
    return sorted_result[::-1], new_results[::-1]

In [94]:
results, results_pred = printResult(results)

In [95]:
df = pd.DataFrame(results_pred, results, columns=["3着以内の確率"])

In [96]:
df

,3着以内の確率
4,0.806819
1,0.724078
7,0.576926
3,0.492589
2,0.398407
5,0.293101
6,0.184295
